In [1]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
!pip install requests
!pip install bs4
!pip install pandas==2.2.0
!pip install fake-useragent==1.5.0
!pip install qdrant_client
!pip install sentence_transformers
!pip install tqdm

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

import numpy as np
import pandas as pd
import torch

from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer

from tqdm.notebook import tqdm

**Парсер**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Задайте диапазон страниц (ваш пример имеет только одну страницу, но это может измениться)
start_page = 3329
end_page = 10000

# Создайте пустой DataFrame для хранения данных
df = pd.DataFrame(columns=['URL', 'Вопрос', 'Ответ' ])

# Цикл по диапазону страниц
for page_number in range(start_page, end_page + 1):
    # Сформируйте URL для каждой страницы
    url = f'http://www.cbr.ru/Reception/TopicalMessage/Page/{page_number}'

    # Отправьте запрос на страницу
    response = requests.get(url)

    # Проверьте успешность запроса и наличие ошибки 404
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Проверьте, что страница не содержит сообщение об ошибке 404
        h1_element = soup.find('h1')
        if h1_element:
            # Извлеките вопрос и ответ
            question = h1_element.text.strip()
            answer_element = soup.find('div', class_='landing-text').find('p')
            answer = answer_element.text.strip() if answer_element else ""

            # Добавьте данные в DataFrame
            df = pd.concat([df, pd.DataFrame({'URL': [url], 'Вопрос': [question], 'Ответ': [answer]})], ignore_index=True)
            print(page_number, " ", end="")
        else:
            print(f"Страница {url} не содержит элемента h1 с классом '_long'. Пропускаем.")


# Выведите полученный DataFrame
df


3331  3338  3339  3342  3343  3344  3357  3400  3401  3402  3403  3404  3405  3406  3410  3411  3412  3413  3415  3416  3417  3418  3419  3420  3421  3423  3426  3427  3428  3432  3445  3446  3447  3450  3451  3452  3454  3456  3457  3458  3459  3460  3461  3462  3463  3464  3466  3467  3470  3471  3472  3474  3475  3476  3501  3502  3503  3504  3505  3506  3507  3508  3509  3510  3511  3512  3513  3514  3515  3516  3517  3518  3519  3520  3521  3522  3523  3526  3527  3528  3530  3535  3536  3537  3538  3539  3540  3541  3542  3543  3544  3545  3546  3547  3548  3549  3550  3551  3552  3553  3554  3555  3556  3557  3558  3559  3563  3564  3565  3566  3567  3568  3569  3570  3572  3573  3578  3580  3582  3583  3584  3585  3586  3587  3589  3590  3591  3592  3595  3596  3598  3599  3600  3601  3602  3603  3604  3605  3606  3607  3609  3610  3611  3612  3613  3614  3617  3618  3619  3620  3621  3623  3624  3625  3628  3629  3630  3631  3632  3633  3634  3635  3636  3637  3638  3639  3640

,URL,Вопрос,Ответ
0,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Следует ли заполнять графу 15 раздела 3 Отчета...,В графе 15 раздела 3 следует указывать код «ДП...
1,http://www.cbr.ru/Reception/TopicalMessage/Pag...,"Необходимо ли направлять сведения, предусмотре...","Сведения, предусмотренные пунктом 4 статьи 10...."
2,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Как определить руководителя структурного подра...,Если организационная структура профессионально...
3,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Какие даты необходимо отражать по показателям ...,В случае представления информации о временном ...
4,http://www.cbr.ru/Reception/TopicalMessage/Pag...,В каком режиме работает электронный документоо...,С 23.01.2019 обмен с кредитными организациями ...
...,...,...,...
4885,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Как рассчитывается минимально возможная сумма ...,Частью 7.1 статьи 6 Закона № 353-ФЗ установлен...
4886,http://www.cbr.ru/Reception/TopicalMessage/Pag...,"В случае, если при заключении договора потреби...",
4887,http://www.cbr.ru/Reception/TopicalMessage/Pag...,При изменении лимита кредитования следует внов...,В соответствии с частью 1.1 статьи 6 Закона № ...
4888,http://www.cbr.ru/Reception/TopicalMessage/Pag...,"Верно ли, что при изменении (увеличении/уменьш...",При изменении договора потребительского кредит...


In [ ]:
# добавляем df к актуальной db
df.to_csv('cbr_db2.csv')


In [ ]:
df2 = pd.read_csv("cbr_db.csv")
df.iloc[:598]

,URL,Вопрос,Ответ
0,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Следует ли заполнять графу 15 раздела 3 Отчета...,В графе 15 раздела 3 следует указывать код «ДП...
1,http://www.cbr.ru/Reception/TopicalMessage/Pag...,"Необходимо ли направлять сведения, предусмотре...","Сведения, предусмотренные пунктом 4 статьи 10...."
2,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Как определить руководителя структурного подра...,Если организационная структура профессионально...
3,http://www.cbr.ru/Reception/TopicalMessage/Pag...,Какие даты необходимо отражать по показателям ...,В случае представления информации о временном ...
4,http://www.cbr.ru/Reception/TopicalMessage/Pag...,В каком режиме работает электронный документоо...,С 23.01.2019 обмен с кредитными организациями ...
...,...,...,...
593,http://www.cbr.ru/Reception/TopicalMessage/Pag...,"Просим пояснить, как на дату начала аренды НФО...","НФО, применяющая пункт 5 статьи 170 НК РФ, при..."
594,http://www.cbr.ru/Reception/TopicalMessage/Pag...,"Какими бухгалтерскими записями НФО, применяюща...",При перечислении суммы арендной платы арендато...
595,http://www.cbr.ru/Reception/TopicalMessage/Pag...,НФО применяет пункт 4 статьи 170 НК РФ. Просим...,Не позднее даты уплаты арендного платежа аренд...
596,http://www.cbr.ru/Reception/TopicalMessage/Pag...,В случае если НФО применяет пункт 4 статьи 170...,"НФО — арендатор, применяющая пункт 4 статьи 17..."


In [ ]:
data = df.copy()
data = df.iloc[:4629]
data.to_csv("cbr_db3.csv")
data.select_dtypes(include=['object']).columns

Index(['URL', 'Вопрос', 'Ответ'], dtype='object')

In [2]:
import json

import numpy as np
import pandas as pd
import torch

from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer

from tqdm.notebook import tqdm

COLLECTION_NAME = "cbr_db"
QDRANT_HOST = "https://592280ba-f0b2-4d9c-a9be-4acf436ac5a7.us-east4-0.gcp.cloud.qdrant.io:6333"
QDRANT_API_KEY = "9cA2MEDlGec7XBh7EtrDtm89JthaTtvp5Nf0lj3wi4nffdV_iYN7iA"


client = QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)

client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
    	size=384,
    	distance=models.Distance.COSINE
    ),
)



True

In [3]:
df = pd.read_csv("cbr_db3.csv")

Модель стандартная MiniLM

In [ ]:
#Этот кал на колабовском процессоре будет обсчитываться вечность
#Надо на видюхе поднимать
model = SentenceTransformer(
    "msmarco-MiniLM-L-6-v3",
    device="cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu",
)

In [ ]:
vectors = np.load("vectors.npy")
vectors

array([[-0.32754108, -0.224761  ,  0.22367987, ..., -0.12076021,
         1.1005245 , -0.76400995],
       [-0.14068028, -0.16344905,  0.18829383, ..., -0.1599939 ,
         1.0558248 , -0.80128956],
       [-0.143603  , -0.17280051,  0.2744521 , ..., -0.05739379,
         1.0692114 , -0.7298214 ],
       ...,
       [-0.0959615 , -0.1672151 ,  0.00897071, ..., -0.31439388,
         1.1868279 , -0.9448043 ],
       [-0.0921165 , -0.5445214 ,  0.27433112, ..., -0.10332508,
         1.107974  , -1.2641233 ],
       [ 0.07169092, -0.04427026, -0.05371436, ..., -0.01527349,
         0.9879756 , -1.2093343 ]], dtype=float32)

In [ ]:

client.upsert(
    collection_name=COLLECTION_NAME,
    points=models.Batch(
        ids=[i for i in range(df.shape[0])],
        payloads=[
            {
                "question": row["Вопрос"],
                "answer": row["Ответ"],
                "url": row["URL"],
            }
            for _, row in df.iterrows()
        ],
        vectors=[v.tolist() for v in vectors],
    ),
)

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to encode text using BERT model
def encode_text(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Iterate through DataFrame and encode 'Вопрос' and 'Ответ', then add to qDrant
for index, row in df.iterrows():
    url = row['URL']
    embedding = encode_text(row['Вопрос'] + " " + row["Ответ"])

    # Create index for qDrant
    index_data = {
        "id": url,
        "question_embedding": embedding.tolist(),
    }

    # Add data to qDrant
    client.index(index_name="your_index_name", index_data=[index_data])

print("Data successfully added to qDrant.")


AttributeError: 'QdrantClient' object has no attribute 'index'

Пробую через фаст-ембединги, но не могу залить всю хуйню на бд, даже локально, инфа отсюда https://qdrant.tech/articles/fastembed/

In [ ]:
from fastembed.embedding import DefaultEmbedding
from typing import List
import numpy as np

# Assuming df is already defined
df["text"] = df['Вопрос'] + df['Ответ']
documents: List[str] = df["text"]


embedding_model = DefaultEmbedding()
embeddings: List[np.ndarray] = list(embedding_model.embed(documents))  # Convert to list

qdrant_address = "http://localhost:6333"
client = QdrantClient(qdrant_address)


# Assuming you have a qDrant client and collection already set up
client.add(
    collection_name=COLLECTION_NAME,
    documents=df['text'].tolist(),
    metadata=df["URL"].tolist(),
    embeddings=embeddings
)

print("Data successfully added to qDrant.")


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/156.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/156.5 kB 1.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.7 MB/s eta 0:00:00


In [8]:
from sentence_transformers import SentenceTransformer


sentences = [
    'query: Что такое круглые тензоры?',
    'passage: Abstract: we introduce a novel method for compressing round tensors based on their inherent radial symmetry. We start by generalising PCA and eigen decomposition on round tensors...',
]

model = SentenceTransformer('d0rj/e5-large-en-ru')
embeddings = model.encode(sentences, convert_to_tensor=True)
embeddings.size()
# torch.Size([2, 1024])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

torch.Size([2, 1024])

In [9]:
embeddings

tensor([[-0.0134,  0.0248, -0.0368,  ..., -0.0515, -0.0024, -0.0045],
        [ 0.0374,  0.0103, -0.0251,  ..., -0.0272, -0.0475, -0.0036]])

In [4]:
torch.cuda.is_available()

True

In [5]:
#Этот кал на колабовском процессоре будет обсчитываться вечность
#Надо на видюхе поднимать
model = SentenceTransformer(
    "d0rj/e5-large-en-ru",
    device="cpu"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [15]:
vectors = []

sentences = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
      question = row["Вопрос"]
      paragraphs = row["Ответ"].split("\n")  # Разделение текста на абзацы
    except:
      continue

    for paragraph in paragraphs:
        # Формирование строки запроса и абзаца с префиксами "query:" и "passage: Abstract:"
        query = "query: " + question
        passage = "passage: Abstract: " + paragraph

        # Добавление строки запроса и абзаца в пакет
        sentences.append(query)
        sentences.append(passage)
        try:
            
            vectors.append(model.encode(sentences))
            sentences = []
        except TypeError:
            print(sentences)
            sentences = []
            continue
        except IndexError:
            sentences = []
            continue


# vectors = np.concatenate(vectors)

with open('vectors_Xenova.npy', 'wb') as f:
    np.save(f, vectors)


  0%|          | 0/4629 [00:00<?, ?it/s]